In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np

from keras import backend
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import TensorBoard, Callback
from keras.layers import Dense, GlobalAveragePooling2D,Dropout
from keras.models import Model
from keras.optimizers import Adam, RMSprop
backend.set_image_data_format('channels_last')

Using TensorFlow backend.


In [2]:
# from classification_models.keras import Classifiers
# inceptionv3, preprocess_input = Classifiers.get('inceptionv3')
import efficientnet.keras as efn
preprocess_input = efn.preprocess_input

In [3]:
input_size = 380

In [4]:
def model_fn(objective, optimizer, metrics):
#     base_model = inceptionv3(include_top=False,
#     base_model = xception(include_top=False,
    base_model = efn.EfficientNetB4(include_top=False,
#     base_model = inceptionresnetv2(include_top=False,
                           input_shape=(input_size, input_size, 3),
                           classes=9,
                           weights='imagenet')
    x = base_model.output
    x = GlobalAveragePooling2D( )(x)
    predictions = Dense(9, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(loss=objective, optimizer=optimizer, metrics=metrics)
    model.summary( )
    return model
optimizer = Adam(lr=0.01)
objective = 'categorical_crossentropy'
metrics = ['accuracy']
model1 = model_fn(objective, optimizer, metrics)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 380, 380, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 190, 190, 48) 1296        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 190, 190, 48) 192         stem_conv[0][0]                  
_______________

In [5]:
model1.load_weights('./model_snapshots/Q0018/model_at_epoch_44.h5')

In [6]:
intermediate_model1 = Model(inputs=model1.input, outputs=[model1.get_layer('global_average_pooling2d_1').output])

In [7]:
from test_data import load_test_data
img_paths,test_data = load_test_data(input_size,preprocess_input)

total img: 4260 
Test data ok!


In [8]:
# from keras.preprocessing.image import ImageDataGenerator
# test_datagen1 = ImageDataGenerator(
#             horizontal_flip=True,
# )
# tta_steps = 5
# bs = 64

In [9]:
# import numpy as np
# predictions = []
# result = []
# for i in range(tta_steps):
#     preds =model1.predict_generator(test_datagen1.flow(test_data, batch_size=bs, shuffle=False), steps = len(test_data)/bs,verbose=1)
#     predictions.append(preds)
# final_pred = np.mean(predictions, axis=0)
# final_pred = np.argmax(final_pred, axis=-1)

In [10]:
# infos = []
# for index, pred_ in enumerate(final_pred):
#     pred_label = pred_ + 1
#     infos.append('%s,%s\n' % (img_paths[index], pred_label))
# path = './bagging/21_A0007_55.csv'
# with open(path, 'w') as f:
#     f.write('﻿FileName,type\n')
#     f.writelines(infos)
# print('end')

In [11]:
from svm_data import load_test_data_train
train_data,train_label = load_test_data_train(input_size,preprocess_input)

total label: 6391 


/usr/local/lib/python3.5/dist-packages/PIL/Image.py:969: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


total img: 6391 
Train data ok!


In [12]:
from keras.preprocessing.image import ImageDataGenerator
test_datagen1 = ImageDataGenerator(
            horizontal_flip=True,
#             width_shift_range=0.01,
#             height_shift_range=0.01,
)
tta_steps = 5

bs = 64
predictions_1 = []
for i in range(tta_steps):
    preds =intermediate_model1.predict_generator(test_datagen1.flow(train_data, batch_size=bs, shuffle=False), steps = len(train_data)/bs,verbose=1)
    predictions_1.append(preds)
intermediate_output = np.mean(predictions_1, axis=0)

100/99 [==============================] - 104s 1s/step


In [13]:
from sklearn.svm import SVC
svm = SVC(kernel='rbf',gamma='auto')
svm.fit(intermediate_output, train_label)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [14]:
tta_steps = 5
bs = 64
predictions_1 = []
for i in range(tta_steps):
    preds =intermediate_model1.predict_generator(test_datagen1.flow(test_data, batch_size=bs, shuffle=False), steps = len(test_data)/bs,verbose=1)
    predictions_1.append(preds)
final_pred = np.mean(predictions_1, axis=0)

67/66 [==============================] - 68s 1s/step


In [15]:
pre_fin = svm.predict(final_pred)

In [16]:
print('###################  wirte ing  ##############################')
infos = []
for index, pred_ in enumerate(pre_fin):
    pred_label = pred_ + 1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
path = './submit/29_Q0018_44_svm.csv'
with open(path, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

###################  wirte ing  ##############################
end


In [18]:
from keras.preprocessing.image import ImageDataGenerator
test_datagen1 = ImageDataGenerator(
            horizontal_flip=True,
)
tta_steps = 5

In [19]:
model2 = model_fn(objective, optimizer, metrics)
model2.load_weights('./model_snapshots/Q0008/model_at_epoch_31.h5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 380, 380, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 189, 189, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 189, 189, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 189, 189, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [20]:
intermediate_model2 = Model(inputs=model2.input, outputs=[model2.get_layer('global_average_pooling2d_2').output])

In [21]:
predictions_1 = []
for i in range(tta_steps):
    preds =intermediate_model2.predict_generator(test_datagen1.flow(train_data, batch_size=bs, shuffle=False), steps = len(train_data)/bs,verbose=1)
    predictions_1.append(preds)
intermediate_output2 = np.mean(predictions_1, axis=0)

100/99 [==============================] - 60s 600ms/step


In [22]:
from sklearn.svm import SVC
svm2 = SVC(kernel='rbf',gamma='auto')
svm2.fit(intermediate_output2, train_label)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [23]:
predictions_1 = []
for i in range(tta_steps):
    preds =intermediate_model2.predict_generator(test_datagen1.flow(test_data, batch_size=bs, shuffle=False), steps = len(test_data)/bs,verbose=1)
    predictions_1.append(preds)
final_pred2 = np.mean(predictions_1, axis=0)

67/66 [==============================] - 35s 527ms/step


In [24]:
pre_fin2 = svm2.predict(final_pred2)

In [25]:
print('###################  wirte ing  ##############################')
infos = []
for index, pred_ in enumerate(pre_fin2):
    pred_label = pred_ + 1
    infos.append('%s,%s\n' % (img_paths[index], pred_label))
path =  './submit/26_Q0008_31_svm.csv'
with open(path, 'w') as f:
    f.write('﻿FileName,type\n')
    f.writelines(infos)
print('end')

###################  wirte ing  ##############################
end
